# Image-Scraper
#### Scrape images from the web

Created by Rutuparn Pawar [InputBlackBoxOutput]

-----

Enter the list of keywords in the cell below


In [ ]:
%%writefile keywords.txt
airplane
banana
cat

Writing keywords.txt


In [ ]:
# Search engine to use while scraping the images. 
search_engine = "all" # [google, bing, yahoo, duckduckgo, all]

# Number of images per keyword. All images are downloaded when set to None
num_images = None

# Add a prefix and/or suffix to all the keywords
prefix = None
suffix = None

# Threshold for removing similar images
similarity_threshold = 0.98

# Output directory to store the scraped images
out_dir = "images"

-----

In [ ]:
!pip install imagehash

     |████████████████████████████████| 812 kB 4.6 MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=530ee189ea7218a38c4f6e3c400d58243472eea6c05f3d6925420fd8b5e8584a
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
Successfully built imagehash


In [ ]:
import os, platform, sys, argparse, glob
import time
import base64
import hashlib
from io import BytesIO
from html.parser import HTMLParser
from urllib.parse import quote, unquote

from tqdm import tqdm
import urllib3
import PIL.Image as Image

import numpy as np
import imagehash

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [ ]:
!pip install selenium
from selenium import webdriver

     |████████████████████████████████| 954 kB 5.1 MB/s 
     |████████████████████████████████| 356 kB 52.4 MB/s 
     |████████████████████████████████| 138 kB 69.3 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 3.5 MB 37.9 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!apt-get update 
!apt install chromium-chromedriver -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [607 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 P

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
# Instantiate and connect to the chrome driver 
def setup_browser():
	options = webdriver.ChromeOptions()
	options.add_argument('--headless')
	options.add_argument('--no-sandbox')
	options.add_argument('--disable-dev-shm-usage')
	browser = webdriver.Chrome('chromedriver', options=options)

	return browser

In [ ]:
# Get the SHA-256 hash of a file
def sha256(fname, size=4096):
 
	sha256_hash = hashlib.sha256()
	with open(fname, 'rb') as f:
		for byte_block in iter(lambda: f.read(4096), b""):
			sha256_hash.update(byte_block)
		
	return sha256_hash.hexdigest()

# Find difference between files using SHA-256 and remove duplicates
def remove_duplicate_images(directory):
	print("\nChecking for duplicate images by comparing SHA-256 hash")
	flag = False

	file_list = glob.glob(f"{directory}/*.png")

	unique = []
	for file in file_list:
		filehash = sha256(file)

		if filehash not in unique:
			unique.append(filehash)
		else:
			print(f"Removing duplicate image: {file}")
			os.remove(file)	
			flag = True
			
	if flag == False:
		print("No duplicate images found")


# Get a combined perceptual hashs of a image
def get_perceptual_hash(img_path):
	img = Image.open(img_path)

	hashes = [
		imagehash.average_hash,
		imagehash.phash,
		imagehash.dhash,
		imagehash.whash,
	]

	combined_hash = np.array([h(img).hash for h in hashes]).flatten()
	combined_hash = np.where(combined_hash==True, 1, 0)

	return combined_hash

# Compare combined perceptual hashs of two images
def compare_hash(hash1, hash2):
	assert len(hash1) == len(hash2)

	count = 0
	for i in range(len(hash1)):
		if hash1[i] == hash2[i]:
			count +=1

	return count/len(hash1)

# Remove similar images using perceptual hashs
def remove_similar_images(directory, similarity_threshold=0.98):
	print("\nChecking for similar images")
	file_list = glob.glob(f"{directory}/*.png")

	found = False
	unique = []
	for file in file_list:
		filehash = get_perceptual_hash(file)

		flag = False
		for each in unique:
			similarity = compare_hash(each, filehash)

			if similarity >= similarity_threshold:
				flag = True
				found = True
				break

		if flag:
			print(f"Removing similar image: {file}")
			os.remove(file)
		else:
			unique.append(filehash)

	if not found:
		print("No similar images found")

In [ ]:

# Class to extract the value of specific HTML tag attribute
class Extractor(HTMLParser):
  src = []
  tag_attr = None
  def handle_starttag(self, tag, attrs):
    if tag == "img":
      for each in attrs:
        if each[0] == self.tag_attr:
          self.src.append(each[1])

# Create output directory if it does not exist
def create_output_directory(keyword, out_dir=None):
	if out_dir == None:
		os.makedirs(keyword, exist_ok=True)		
	else:
		os.makedirs(out_dir, exist_ok=True)
		os.makedirs(f"{out_dir}/{keyword}", exist_ok=True)

def add_prefix_suffix(keyword, prefix=None, suffix=None):
	if prefix != None:
		keyword = prefix + " " + keyword

	if suffix != None:
		keyword = keyword + " " + suffix
	
	return keyword

def filter_src_format(src_list):
	filtered = []

	for each_src in src_list:
		if ".png" in each_src or ".jpg" in each_src or ".jpeg" in each_src:
			filtered.append(each_src)
		elif "/png" in each_src or "/jpg" in each_src or "/jpeg" in each_src:
			filtered.append(each_src)
		elif "https:" in each_src:
			filtered.append(each_src)
		else:
			continue

	return filtered


def get_img_data(url, src):
	if "https:" in src or "www." in src:
		response = http.request('GET', src)
		img_data = BytesIO(response.data)

	elif src.endswith(".png") or src.endswith(".jpg"):
		base_url = urlparse(url).netloc
		src = base_url + src

		response = http.request('GET', src)
		img_data = BytesIO(response.data)

	else:
		src = src.split(',')[-1]
		img_data = base64.b64decode(src)
		img_data = BytesIO(img_data)

	return img_data

# Search for the specified keyword using the specified search engine, load the url on chrome 
# using chromedriver, extract certain attribute values and then collect the images
def scrape_images_search_engine(keyword, search_engine, output_directory, num_images=None):		
	print(f"\nSearch engine: {search_engine}")

	search_engine_urls = {
		"google" : f"https://www.google.com/search?tbm=isch&q={quote(keyword)}",
		"bing" : f"https://www.bing.com/images/search?q={quote(keyword)}",
		"yahoo" : f"https://images.search.yahoo.com/search/images?p={quote(keyword)}",
		"duckduckgo": f"https://duckduckgo.com/?q={quote(keyword)}&iax=images&ia=images"
	}
	url = search_engine_urls[search_engine]
	print(f"URL: {url}")

	browser.get(url)
	time.sleep(2)

	scroll_count = {"google": 3, "bing": 3, "yahoo": 1, "duckduckgo": 5}
	for _ in range(scroll_count[search_engine]):
		browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
		browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
		time.sleep(2)

	extractor.src = []

	if search_engine == "google":
		extractor.tag_attr = "data-src"
		extractor.feed(browser.page_source)
		
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

	elif search_engine == "bing":
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

		extractor.src = list(map(lambda x: x.split('?w')[0], extractor.src))
		extractor.src = list(set(extractor.src))

		filtered = []
		for each in extractor.src:
			if "OIP" in each:
				filtered.append(each)

		extractor.src = filtered	
		
	elif search_engine == "yahoo":
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

		extractor.src = list(map(lambda x: x.split('&')[0], extractor.src))
		extractor.src = list(set(extractor.src))

	elif search_engine == "duckduckgo":
		extractor.tag_attr = "src"
		extractor.feed(browser.page_source)

		extractor.src = list(map(lambda x: unquote(x.split('?')[-1][2:]), extractor.src))

	extractor.src = filter_src_format(extractor.src)
	len_src = len(extractor.src)
	print(f"Number of images found: {len_src}")

	if num_images != None:
		src_list = extractor.src[:num_images]
	else:
		src_list = extractor.src

	count = 0
	for each_src in tqdm(src_list):
		try:
			img_data = get_img_data(url, each_src)

			image = Image.open(img_data).convert("RGBA")
			image.save(f"{output_directory}/{search_engine[0]}-{count+1}.png")
				
			count+=1

		except:
			print(f"Something went wrong while scraping the image at URL:\n{each_src}")

	if num_images != None:
		print(f"Downloaded {count}/{num_images} images")
	else:
		print(f"Downloaded {count}/{len_src} images")

In [ ]:
browser = setup_browser()
extractor = Extractor()
http = urllib3.PoolManager()

if search_engine not in ['google', 'bing', 'yahoo', 'duckduckgo', 'all']:
  raise Exception("Search engine needs to be one of the following: google, bing, yahoo, duckduckgo or all")

if num_images != None:
  num_images = [num_images // 4 + (1 if x < num_images % 4 else 0)  for x in range (4)]

with open("keywords.txt", 'r') as infile:
  keywords = infile.read().splitlines()

  for each in keywords:
    if each != "":
      print('\n' + '-' * 100)
      
      keyword = add_prefix_suffix(each, prefix=prefix, suffix=suffix).strip()
      print(f"Keyword: {keyword}")

      create_output_directory(keyword, out_dir)
      output_directory = keyword if out_dir == None else f"{out_dir}/{keyword}"

      if search_engine != "all":
        scrape_images_search_engine(keyword=keyword, search_engine=search_engine, output_directory=output_directory, num_images=num_images)
      else:
        if num_images != None:
          for i, each_se in enumerate(['google', 'bing', 'yahoo', 'duckduckgo']):
            scrape_images_search_engine(keyword=keyword, search_engine=each_se, output_directory=output_directory, num_images=num_images[i])
        else:
          for each_se in ['google', 'bing', 'yahoo', 'duckduckgo']:
            scrape_images_search_engine(keyword=keyword, search_engine=each_se, output_directory=output_directory, num_images=num_images)

      remove_duplicate_images(output_directory)
      remove_similar_images(output_directory, similarity_threshold)
      print('-' * 100)
    
      time.sleep(2)

browser.quit()


----------------------------------------------------------------------------------------------------
Keyword: airplane

Search engine: google
URL: https://www.google.com/search?tbm=isch&q=airplane
Number of images found: 478


100%|██████████| 478/478 [00:09<00:00, 50.28it/s]


Downloaded 478/478 images

Search engine: bing
URL: https://www.bing.com/images/search?q=airplane
Number of images found: 114


100%|██████████| 114/114 [00:13<00:00,  8.67it/s]


Downloaded 114/114 images

Search engine: yahoo
URL: https://images.search.yahoo.com/search/images?p=airplane
Number of images found: 113


...

Downloaded 113/113 images

Search engine: yahoo
URL: https://images.search.yahoo.com/search/images?p=cat
Number of images found: 120


100%|██████████| 120/120 [00:16<00:00,  7.29it/s]


Downloaded 120/120 images

Search engine: duckduckgo
URL: https://duckduckgo.com/?q=cat&iax=images&ia=images
Number of images found: 594


100%|██████████| 594/594 [00:58<00:00, 10.12it/s]


Downloaded 594/594 images

Checking for duplicate images by comparing SHA-256 hash
Removing duplicate image: images/cat/y-116.png
Removing duplicate image: images/cat/b-35.png
Removing duplicate image: images/cat/y-66.png
Removing duplicate image: images/cat/g-355.png
Removing duplicate image: images/cat/y-92.png
Removing duplicate image: images/cat/y-81.png
Removing duplicate image: images/cat/y-39.png
Removing duplicate image: images/cat/b-11.png
Removing duplicate image: images/cat/b-22.png
Removing duplicate image: images/cat/b-2.png
Removing duplicate image: images/cat/b-95.png
Removing duplicate image: images/cat/g-431.png
Removing duplicate image: images/cat/g-356.png
Removing duplicate image: images/cat/d-127.png
Removing duplicate image: images/cat/b-97.png
Removing duplicate image: images/cat/d-87.png
Removing duplicate image: images/cat/g-354.png
Removing duplicate image: images/cat/b-48.png
Removing duplicate image: images/cat/g-472.png
Removing duplicate image: images/cat/

In [ ]:
!zip images.zip images/*/*
!rm -rf images

In [ ]:
# !cp images.zip /path/to/directory/images.zip